Neste trabalho iremos fazer a busca dos melhores hiperparametros de uma SVM para Regressão num banco de dados em particular. Para o desenvolvimento utilizaremos a biblioteca sklearn.svm que apresenta a função  SVR() que implementa o regressor SVM e tem vários hiperparametros.
Vamos usar o kernel “rbf”, havendo 3 hiperparametros que consideramos como os mais importantes: C, gamma, e epsilon.

Vamos fazer a busca no range:
    - C entre 2^{-5} e 2^{15}  (uniforme nos expoentes);
    - gamma entre 2^{15} e 2^{-3} (uniforme nos expoentes);
    - episolon entre 0.05 e 10  (uniforme neste intervalo);
    
Utilizamos como dados de treino e testes os arquivos Xtreino5.npy e Xteste5.npy. Para as sáidas dos dados correspondenetres usamos os arquivos ytreino5.npy e yteste5.npy. Todos os arquivos foram disponibilizados na página no trabalho https://www.ic.unicamp.br/~wainer/cursos/1s2020/431/ex4.html.

### Importação das Bibliotecas

Como primeiro passo, importamos as bibliotecas necessárias para a implementação desse trabalho: numpy e sklearn.svm.

In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import metrics
import pyspark
from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score

from pyswarm import pso
from simanneal import Annealer
import optuna
from scipy.stats import loguniform
import random

### Leitura e Exibição dos arquivos

Após importamos as bibliotecas necessárias, realizamos a leitura dos arquivos .npy disponibilizados para uso neste trabalho. Para isso fizemos uso da função load() da biblioteca numpy. Armazenamos cada um dos arquivos de treino e de teste em variáveis locais.

In [2]:
x_treino = np.load('data/Xtreino5.npy')
x_teste = np.load('data/Xteste5.npy')
y_treino = np.load('data/ytreino5.npy')
y_teste = np.load('data/yteste5.npy')

### Medida de erro

Como primeira tarefa, para cada conjunto de hiperparametros, treinamos o SVM no conjunto de treino (Xtreino e ytreino), e medimos o erro absoluto médio (MAE) no conjunto de teste (Xteste e yteste).

## Randomized Search

In [3]:
n_combinations = 125

# Hyperparameters Search Space
# C_range = np.random.uniform(-5, 15, n_combinations).astype(float)
# C_range = 2**C_range

C_range = loguniform(2**-5, 2**15).rvs(size=n_combinations)

# gamma_range = np.random.uniform(-15, 3, n_combinations).astype(float)
# gamma_range = 2**gamma_range

gamma_range = loguniform(2**-15, 2** 3).rvs(size=n_combinations)

epsilon_range = np.random.uniform(0.05, 1.0, n_combinations).astype(float)

 
hyperparameters = {'gamma': list(gamma_range), 
                    'C': list(C_range),
                  'epsilon': list(epsilon_range)}
 
# print (hyperparameters)

In [4]:
# Run randomized search
randomCV = RandomizedSearchCV(SVR(kernel='rbf'), param_distributions=hyperparameters, n_iter=20)
randomCV.fit(x_treino, y_treino)
 
# Identify optimal hyperparameter values
best_gamma  = randomCV.best_params_['gamma']
best_C      = randomCV.best_params_['C']
best_epsilon= randomCV.best_params_['epsilon']

print("The best performing C value is: {:5.2f}".format(best_C))
print("The best performing gamma value is: {:5.5f}".format(best_gamma))
print("The best performing epsilon value is: {:5.2f}".format(best_epsilon))


The best performing C value is: 62.85
The best performing gamma value is: 0.00016
The best performing epsilon value is:  0.93


In [5]:
# Validation

svr  = SVR(kernel='rbf', gamma=best_gamma, epsilon=best_epsilon, C=best_C)
svr.fit(x_treino, y_treino)

pred = svr.predict(x_teste)

# print(regression.score(x_teste, y_teste))
print("MAE: ", metrics.mean_absolute_error(y_true=y_teste, y_pred=pred))

MAE:  3.674047769591726


### Grid seach

O próximo algoritimo de otimização proposto foi o Grid Search. Seguindo a especificação de uma busca em uma grid de 5x5x5, amostras do range de busca foram tomadas:

In [6]:
grid_c = random.sample(list(C_range), k=5)
grid_gamma = random.sample(list(gamma_range), k=5)
grid_episolon = random.sample(list(epsilon_range), k=5)

hyperparameters_grid = {'gamma': list(grid_gamma), 
                        'C': list(grid_c),
                        'epsilon': list(grid_episolon)}

In [7]:
# Run Grid Search
randomCV = GridSearchCV(SVR(kernel='rbf'), param_grid=hyperparameters_grid, cv = 5)
randomCV.fit(x_treino, y_treino)
 
# Identify optimal hyperparameter values
best_gamma  = randomCV.best_params_['gamma']
best_C      = randomCV.best_params_['C']
best_epsilon= randomCV.best_params_['epsilon']
 
print("The best performing gamma value is: {:5.5f}".format(best_gamma))
print("The best performing C value is: {:5.2f}".format(best_C))
print("The best performing epsilon value is: {:5.2f}".format(best_epsilon))

The best performing gamma value is: 0.00005
The best performing C value is: 1356.98
The best performing epsilon value is:  0.09


In [8]:
# Validation

svr  = SVR(kernel='rbf', gamma=best_gamma, epsilon=best_epsilon, C=best_C)
svr.fit(x_treino, y_treino)

pred = svr.predict(x_teste)

# print(regression.score(x_teste, y_teste))
print("MAE: ", metrics.mean_absolute_error(y_true=y_teste, y_pred=pred))

MAE:  2.9099338966940835


### Otimização bayesiana

Outro algoritimo proposto foi a otimização bayesiana. Para a sua implementação utilziamos a biblioteca hyperopt, que dispnibiliza O regressor (TPE) para modelar a distribuição de probabilidades que é muito mais rápido que a implementação padrão utilizando “processos gaussianos”.

Esta implementação foi feita em passos:

### Definir a Função de mínimo

Como queremos pesquisar por Support Vector Machines (SVM), definimos um parâmetro params ['type'] como o nome do modelo, e uma função para executar o treinamento e retornar a precisão da validação cruzada. 
Como estamos tentando maximizar a precisão da validação cruzada, devemos negar esse valor para o hyperopt, pois o hyperopt sabe apenas como minimizar uma função.

In [9]:
def get_acc_status(clf,X_,y):
    acc = cross_val_score(clf, X_, y, cv=5).mean()
    return {'loss': -acc, 'status': STATUS_OK}

In [10]:
def objective(params):
    classifier_type = params['type']
    del params['type']
    if classifier_type == 'svm':
        clf = SVR(**params)
    else:
        return 0
    accuracy = cross_val_score(clf, x_treino, y_treino).mean()
    
    return {'loss': -accuracy, 'status': STATUS_OK}

### Definir espaço de pesquisa sobre os hiperparâmetros

In [11]:
search_space = hp.choice('classifier_type', [
    {
        'type': 'svm',
        'C': hp.uniform('C', (2**-5), (2**15)),
        'gamma': hp.uniform('gamma', (2**-15), (2**3)),
        'epsilon': hp.uniform('epsilon', 0.05, 1.0),
        'kernel': hp.choice('kernel', ['rbf'])
    },
])

### Selecionar um algoritimo de busca

As duas opções principais de algoritmos de busca são:

    - hyperopt.tpe.suggest: Estimadores da Árvore de Parzen, uma abordagem bayesiana que seleciona iterativa e adaptativamente novas configurações de hiperparâmetro para explorar com base em resultados anteriores;
    - hyperopt.rand.suggest: Pesquisa aleatória, uma abordagem não adaptativa que mostra o espaço de pesquisa.
    
Conforme pedido, utilziamos o algoritimo TPE.

### Executar o algoritmo de ajuste com hyperopt fmin ()

Definimos max_evals como o número máximo de pontos no espaço do hiperparâmetro para testar, ou seja, o número máximo de modelos para ajustar e avaliar.

O SparkTrials usa 2 argumentos:
    - parallelism: Número de modelos para ajustar e avaliar simultaneamente.
    - timeout: tempo máximo (em segundos) que fmin pode demorar. Este argumento é opcional.

O rastreamento automatizado do MLflow está ativado por padrão. Ligue para mlflow.start_run () antes de chamar fmin (), como mostra o exemplo abaixo.

In [12]:
hypopt_trials = Trials()
 
best_params = fmin(objective, search_space, algo=tpe.suggest, 
max_evals=125, trials= hypopt_trials)
 
best_gamma  = best_params['gamma']
best_C      = best_params['C']
best_epsilon= best_params['epsilon']
 
print("The best performing gamma value is: {:5.5f}".format(best_gamma))
print("The best performing C value is: {:5.2f}".format(best_C))
print("The best performing epsilon value is: {:5.2f}".format(best_epsilon))

100%|██████████| 125/125 [00:11<00:00, 10.47trial/s, best loss: -0.517901045072695]  
The best performing gamma value is: 0.00359
The best performing C value is: 25984.93
The best performing epsilon value is:  0.44


In [13]:
# Validation

svr  = SVR(kernel='rbf', gamma=best_gamma, epsilon=best_epsilon, C=best_C)
svr.fit(x_treino, y_treino)

pred = svr.predict(x_teste)

# print(regression.score(x_teste, y_teste))
print("MAE: ", metrics.mean_absolute_error(y_true=y_teste, y_pred=pred))

MAE:  4.452631232325906


## PSO

In [14]:
# PARAMETERS
C_MIN = 2**(-5)
C_MAX = 2**15

GAMMA_MIN = 2**(-15)
GAMMA_MAX = 2**3

EPSILON_MIN = 0.05
EPSILON_MAX = 1.0

lb = np.array([C_MIN, GAMMA_MIN, EPSILON_MIN])
ub = np.array([C_MAX, GAMMA_MAX, EPSILON_MAX])

# FUNCTION
def svr_fun(X):
    c = X[0]
    g = X[1]
    eps = X[2]
    
    svr  = SVR(kernel='rbf', C=c, gamma=g, epsilon=eps)
    svr.fit(x_treino, y_treino)
    
    pred = svr.predict(x_teste)
    mae = metrics.mean_absolute_error(y_true=y_teste, y_pred=pred)
    
    return mae

print("PSO...")
x_opt, y_opt = pso(svr_fun, lb, ub, swarmsize=11, maxiter=11)

print(" C optimal: "+ str(x_opt[0])+
     "\n Gamma Optimal: "+ str(x_opt[1])+
     "\n Epsilon Optimal: "+ str(x_opt[2]))
print("MAE: ", str(y_opt))

PSO...
Stopping search: maximum iterations reached --> 11
 C optimal: 14740.885607664062
 Gamma Optimal: 3.0517578125e-05
 Epsilon Optimal: 0.3875906008160747
MAE:  2.329532586890168


## Simulated Annealing

## CMA-ES

In [15]:
def objective(trial):
    c = trial.suggest_loguniform('c', 2**(-5),  2**15)
    gamma = trial.suggest_loguniform('gamma', 2**(-15),  2**3)    
    epsilon = trial.suggest_uniform('epsilon', 0.05,  1.0)
    
    svr  = SVR(kernel='rbf', C=c, gamma=gamma, epsilon=epsilon)
    svr.fit(x_treino, y_treino)
    
    pred = svr.predict(x_teste)
    mae = metrics.mean_absolute_error(y_true=y_teste, y_pred=pred)
    
    return mae

study = optuna.create_study()
study.optimize(objective, n_trials=125)


[I 2020-05-06 07:22:07,226] Finished trial#0 with value: 5.718012616136094 with parameters: {'gamma': 3.7014720077567342, 'c': 2.2665568624495847, 'epsilon': 0.6068782513078113}. Best is trial#0 with value: 5.718012616136094.
[I 2020-05-06 07:22:07,326] Finished trial#1 with value: 3.5601416967391346 with parameters: {'gamma': 0.0011221996279333647, 'c': 80.64333574640082, 'epsilon': 0.14172352299424795}. Best is trial#1 with value: 3.5601416967391346.
[I 2020-05-06 07:22:08,262] Finished trial#2 with value: 4.599421181772981 with parameters: {'gamma': 0.00046810208629241746, 'c': 9419.638664343644, 'epsilon': 0.9797404324762193}. Best is trial#1 with value: 3.5601416967391346.
[I 2020-05-06 07:22:08,334] Finished trial#3 with value: 3.620044030135721 with parameters: {'gamma': 0.0002778424802327442, 'c': 28.65018342247947, 'epsilon': 0.9307057009048231}. Best is trial#1 with value: 3.5601416967391346.
[I 2020-05-06 07:22:08,398] Finished trial#4 with value: 5.7057750787365835 with par

[I 2020-05-06 07:22:25,570] Finished trial#34 with value: 4.355703833769283 with parameters: {'gamma': 0.0013663595774014532, 'c': 14067.249821070534, 'epsilon': 0.6654359364755511}. Best is trial#16 with value: 2.5753514563623803.
[I 2020-05-06 07:22:25,881] Finished trial#35 with value: 3.11177603047703 with parameters: {'gamma': 0.00040312886590855666, 'c': 1210.8297550583984, 'epsilon': 0.4887876921243496}. Best is trial#16 with value: 2.5753514563623803.
[I 2020-05-06 07:22:26,578] Finished trial#36 with value: 2.5562672440821483 with parameters: {'gamma': 6.713914023986103e-05, 'c': 5449.940960958966, 'epsilon': 0.36565326145965776}. Best is trial#36 with value: 2.5562672440821483.
[I 2020-05-06 07:22:27,552] Finished trial#37 with value: 2.4519686300934933 with parameters: {'gamma': 5.645873601532198e-05, 'c': 6148.7611789681, 'epsilon': 0.1781936974787613}. Best is trial#37 with value: 2.4519686300934933.
[I 2020-05-06 07:22:28,482] Finished trial#38 with value: 2.5472759388996

[I 2020-05-06 07:23:12,257] Finished trial#68 with value: 4.149400205778916 with parameters: {'gamma': 4.651131196417641e-05, 'c': 7.125773203541027, 'epsilon': 0.9181306168577806}. Best is trial#54 with value: 2.4008977098957835.
[I 2020-05-06 07:23:14,100] Finished trial#69 with value: 3.554715829164987 with parameters: {'gamma': 0.0001659793959413781, 'c': 16368.849325884023, 'epsilon': 0.7975288332361459}. Best is trial#54 with value: 2.4008977098957835.
[I 2020-05-06 07:23:14,297] Finished trial#70 with value: 2.9176369979964014 with parameters: {'gamma': 9.039657702072606e-05, 'c': 1348.1473180467551, 'epsilon': 0.9950432758142433}. Best is trial#54 with value: 2.4008977098957835.
[I 2020-05-06 07:23:16,353] Finished trial#71 with value: 2.488431785515231 with parameters: {'gamma': 3.098288139051743e-05, 'c': 28199.047029373713, 'epsilon': 0.8477433083638157}. Best is trial#54 with value: 2.4008977098957835.
[I 2020-05-06 07:23:17,215] Finished trial#72 with value: 2.532049811500

[I 2020-05-06 07:24:04,965] Finished trial#102 with value: 2.4115188486434693 with parameters: {'gamma': 4.04467357615737e-05, 'c': 25161.863137888027, 'epsilon': 0.6044605150380016}. Best is trial#54 with value: 2.4008977098957835.
[I 2020-05-06 07:24:07,960] Finished trial#103 with value: 2.5956671494586216 with parameters: {'gamma': 6.739598352498436e-05, 'c': 24728.11439962013, 'epsilon': 0.6143921981924532}. Best is trial#54 with value: 2.4008977098957835.
[I 2020-05-06 07:24:11,002] Finished trial#104 with value: 2.4055954653150904 with parameters: {'gamma': 3.787579790066236e-05, 'c': 32357.341301789784, 'epsilon': 0.5854429806111348}. Best is trial#54 with value: 2.4008977098957835.
[I 2020-05-06 07:24:14,428] Finished trial#105 with value: 2.4153958065455745 with parameters: {'gamma': 3.9145702231048015e-05, 'c': 32723.873092543137, 'epsilon': 0.6454990981164679}. Best is trial#54 with value: 2.4008977098957835.
[I 2020-05-06 07:24:14,512] Finished trial#106 with value: 4.3789

Logo, como resultado os melhores parâmetros encontrados durante a busca são mostrados:

In [16]:
study.best_params

{'c': 11543.43684801506,
 'epsilon': 0.49201377274029967,
 'gamma': 3.0982114295492385e-05}

Bem como o resultado para o  erro absoluto médio (MAE) no conjunto de teste com esses parâmetros:

In [17]:
study.best_value

2.349490664032069